1. adding time decay factor to the features 

In [12]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# 1. Define a dictionary named `data` to store the data for 10 customers across the 7 chosen metrics.
data = {
    'Discount Usage Frequency (DUF)': np.random.rand(10),
    'Discount Spend Ratio (DSR)': np.random.rand(10),
    'Discount items Proportion': np.random.rand(10),
    'Purchase Frequency During Sales (PFDS)': np.random.rand(10),
    'ADA (Savings/Overall Spending)': np.random.rand(10),
    'Purchase Frequency (PF)': np.random.randint(1, 10, 10),
    'Full Price Purchase Frequency': np.random.rand(10)
}

# 2. Create a DataFrame `df` from the dictionary.
df = pd.DataFrame(data)

# 3. Add a 'Customer ID' column with values from 1 to 10.
df.insert(0, 'Customer ID', range(1, 11))

# 4. Add an 'Order Date' column with random dates between '2023-01-01' and '2024-07-28'.
start_date = datetime.strptime('2023-01-01', '%Y-%m-%d')
end_date = datetime.strptime('2024-07-28', '%Y-%m-%d')
df['Order Date'] = [
    start_date + timedelta(days=np.random.randint((end_date - start_date).days))
    for _ in range(10)
]

# 5. Calculate 'Days Since Most Recent Purchase' by subtracting each 'Order Date' from '2024-07-28' and extracting the number of days.
df['Days Since Most Recent Purchase'] = (end_date - df['Order Date']).dt.days

# Calculate Weighted Metrics
def calculate_weighted_metrics(df):
    # Calculate Time Decay Weight (exponential decay with lambda = 0.5)
    df['Time Decay Weight'] = np.exp(
        -0.005* df['Days Since Most Recent Purchase']
    )

    # multiply each metric by the Time Decay Weight to calculate the weighted metric
    for col in [k for k in data.keys()]:
        df[f'{col} Weighted'] = df[col] * df['Time Decay Weight']

    return df

# Apply weighted metrics calculation to the dataframe
df = calculate_weighted_metrics(df)


# Merge the weighted averages back into the original DataFrame.
# df = df.merge(df_new, on='Customer ID', how='left')
# df_new
# Display the first 10 rows of the DataFrame

,Customer ID,Discount Usage Frequency (DUF),Discount Spend Ratio (DSR),Discount items Proportion,Purchase Frequency During Sales (PFDS),ADA (Savings/Overall Spending),Purchase Frequency (PF),Full Price Purchase Frequency,Order Date,Days Since Most Recent Purchase,Time Decay Weight,Discount Usage Frequency (DUF) Weighted,Discount Spend Ratio (DSR) Weighted,Discount items Proportion Weighted,Purchase Frequency During Sales (PFDS) Weighted,ADA (Savings/Overall Spending) Weighted,Purchase Frequency (PF) Weighted,Full Price Purchase Frequency Weighted
0,1,0.916230,0.124489,0.001877,0.640297,0.854861,2,0.655987,2024-05-25,64,0.726149,0.665319,0.090398,0.001363,0.464951,0.620757,1.452298,0.476344
1,2,0.371449,0.783734,0.897147,0.914496,0.967546,9,0.891910,2023-12-01,240,0.301194,0.111878,0.236056,0.270216,0.275441,0.291419,2.710748,0.268638
2,3,0.625895,0.848465,0.556488,0.624642,0.261672,3,0.132962,2023-04-06,479,0.091173,0.057065,0.077357,0.050736,0.056950,0.023857,0.273518,0.012123
3,4,0.206302,0.366328,0.449895,0.615594,0.338453,3,0.156249,2024-02-08,171,0.425283,0.087737,0.155793,0.191333,0.261802,0.143938,1.275850,0.066450
4,5,0.867353,0.261711,0.331050,0.859390,0.160123,9,0.169302,2023-10-31,271,0.257947,0.223731,0.067508,0.085393,0.221677,0.041303,2.321526,0.043671
5,6,0.376988,0.125430,0.591513,0.758312,0.201615,7,0.292462,2023-01-30,545,0.065546,0.024710,0.008221,0.038771,0.049704,0.013215,0.458823,0.019170
6,7,0.486712,0.144339,0.309385,0.707940,0.194453,4,0.531387,2023-12-12,229,0.318224,0.154883,0.045932,0.098454,0.225283,0.061880,1.272896,0.169100
7,8,0.804328,0.347755,0.047082,0.076838,0.669130,7,0.396828,2023-06-08,416,0.124930,0.100485,0.043445,0.005882,0.009599,0.083595,0.874511,0.049576
8,9,0.616712,0.795991,0.565874,0.922004,0.018367,8,0.268965,2023-03-27,489,0.086726,0.053485,0.069033,0.049076,0.079962,0.001593,0.693809,0.023326
9,10,0.347947,0.237712,0.471439,0.392163,0.705321,3,0.179161,2023-07-24,370,0.157237,0.054710,0.037377,0.074128,0.061663,0.110903,0.471711,0.028171


2. Calculating the Discount hunter score

In [13]:
# Define the metric weights dictionary
metric_weights = {
    'Discount Usage Frequency (DUF)': 0.7,
    'Discount Spend Ratio (DSR)': 0.5,
    'Discount items Proportion': 0.4,
    'Purchase Frequency During Sales (PFDS)': 0.5,
    'ADA (Savings/Overall Spending)': 0.4,
    'Full Price Purchase Frequency': -0.4  # Negative weight as it indicates non-discount behavior
}

# Filter the dataframe to include only the relevant columns
df_filtered = df[['Customer ID'] + list(metric_weights.keys())]

# Normalize the metric columns
for metric in metric_weights.keys():
    df_filtered[metric] = (df_filtered[metric] - df_filtered[metric].min()) / (df_filtered[metric].max() - df_filtered[metric].min())

# Multiply each normalized metric by its weight
for metric, weight in metric_weights.items():
    df_filtered[metric] = df_filtered[metric] * weight

# Calculate the Discount Hunter Score
df_filtered['Discount Hunter Score'] = df_filtered[list(metric_weights.keys())].sum(axis=1)

# Merge the Discount Hunter Score back into the original dataframe
df = df.merge(df_filtered[['Customer ID', 'Discount Hunter Score']], on='Customer ID')

# Display the first 10 rows of the updated dataframe
print(df[['Customer ID', 'Discount Hunter Score'] + list(metric_weights.keys())].head(10).to_markdown(index=False, numalign="left", stralign="left"))


| Customer ID   | Discount Hunter Score   | Discount Usage Frequency (DUF)   | Discount Spend Ratio (DSR)   | Discount items Proportion   | Purchase Frequency During Sales (PFDS)   | ADA (Savings/Overall Spending)   | Full Price Purchase Frequency   |
|:--------------|:------------------------|:---------------------------------|:-----------------------------|:----------------------------|:-----------------------------------------|:---------------------------------|:--------------------------------|
| 1             | 1.1274                  | 0.564935                         | 0.376279                     | 0.0619262                   | 0.465384                                 | 0.650774                         | 0.0288511                       |
| 2             | 1.07578                 | 0.351523                         | 0.593379                     | 0.758702                    | 0.421206                                 | 0.964427                         | 0.940283                  

C:\Users\aakas_i4bpqzq\AppData\Local\Temp\ipykernel_21284\2252282888.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[metric] = (df_filtered[metric] - df_filtered[metric].min()) / (df_filtered[metric].max() - df_filtered[metric].min())
C:\Users\aakas_i4bpqzq\AppData\Local\Temp\ipykernel_21284\2252282888.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[metric] = df_filtered[metric] * weight
C:\Users\aakas_i4bpqzq\AppData\Local\Temp\ipykernel_21284\2252282888.py:23: SettingWith

3. Normalization of score between 0-100

In [15]:
#normalize column Discount Hunter Score between 0 and 100
df['Discount Hunter Score'] = (df['Discount Hunter Score'] - df['Discount Hunter Score'].min()) / (df['Discount Hunter Score'].max() - df['Discount Hunter Score'].min()) * 100
df

,Customer ID,Discount Usage Frequency (DUF),Discount Spend Ratio (DSR),Discount items Proportion,Purchase Frequency During Sales (PFDS),ADA (Savings/Overall Spending),Purchase Frequency (PF),Full Price Purchase Frequency,Order Date,Days Since Most Recent Purchase,Time Decay Weight,Discount Usage Frequency (DUF) Weighted,Discount Spend Ratio (DSR) Weighted,Discount items Proportion Weighted,Purchase Frequency During Sales (PFDS) Weighted,ADA (Savings/Overall Spending) Weighted,Purchase Frequency (PF) Weighted,Full Price Purchase Frequency Weighted,Discount Hunter Score
0,1,0.564935,0.376279,0.061926,0.465384,0.650774,4,0.028851,2024-06-30,28,0.869358,0.491131,0.327121,0.053836,0.404586,0.565756,3.477433,0.025082,46.966425
1,2,0.351523,0.593379,0.758702,0.421206,0.964427,8,0.940283,2024-03-24,126,0.532592,0.187218,0.316029,0.404078,0.224331,0.513646,4.260734,0.500787,42.520286
2,3,0.663430,0.383731,0.790038,0.524654,0.426027,1,0.876917,2024-06-13,45,0.798516,0.529760,0.306415,0.630858,0.418944,0.340190,0.798516,0.700233,44.688693
3,4,0.098141,0.538672,0.284060,0.911586,0.351278,2,0.250590,2023-12-04,237,0.305746,0.030006,0.164697,0.086850,0.278714,0.107402,0.611492,0.076617,22.687731
4,5,0.781153,0.066290,0.482659,0.046197,0.564622,4,0.081793,2023-07-14,380,0.149569,0.116836,0.009915,0.072191,0.006910,0.084450,0.598274,0.012234,40.241439
5,6,0.128319,0.875199,0.968782,0.172518,0.994244,8,0.625705,2023-06-22,402,0.133989,0.017193,0.117267,0.129806,0.023115,0.133217,1.071909,0.083837,45.764955
6,7,0.488091,0.933324,0.299824,0.898533,0.855084,1,0.218206,2024-03-31,119,0.551563,0.269213,0.514786,0.165372,0.495597,0.471632,0.551563,0.120354,100.000000
7,8,0.184890,0.245273,0.567549,0.736662,0.520416,1,0.921512,2024-03-12,138,0.501576,0.092737,0.123023,0.284669,0.369492,0.261028,0.501576,0.462208,0.000000
8,9,0.289600,0.652759,0.674515,0.035957,0.209096,2,0.176148,2023-06-19,405,0.131994,0.038225,0.086160,0.089032,0.004746,0.027599,0.263988,0.023250,13.598440
9,10,0.582097,0.114691,0.217687,0.786058,0.910396,8,0.728940,2024-02-05,174,0.418952,0.243871,0.048050,0.091200,0.329320,0.381412,3.351612,0.305391,42.106025
